<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%9A%D0%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Конечные автоматы Мили и Мура
Вариант 5

In [38]:
import numpy as np
import pandas as pd
import random
import copy
from prettytable import PrettyTable

In [39]:
def equiv(a,b): # ~
  rez = 0
  if((a == 1 and b == 1) or (a == 0 and b == 0)): rez = 1
  return rez

def implic(a,b): # ->
  rez = 1
  if(a == 1 and b == 0): rez = 0
  return rez

def pier(a,b): # ^
  rez = 0
  if(a == 0 and b == 0): rez = 1
  return rez

def schaeffer(a,b): # |
  rez = 1
  if(a == 1 and b == 1): rez = 0
  return rez

##Построение таблицы истинности:

In [40]:
#таблица истинности
def truthTable(p_inp, p_st, x, v):
  inner_table = [[(str(x1(p_st[i], p_inp[j])) + str(x2(p_st[i], p_inp[j]))) for j in range(len(p_inp))] for i in range(len(p_st))]

  print('Таблица истинности для автомата Мили')
  table = PrettyTable()
  columns = p_st.copy()
  columns.insert(0, "v \ x")
  table.field_names = columns

  for i in range(len(p_inp)):
    table.add_row([p_inp[i], inner_table[i][0], inner_table[i][1],
                             inner_table[i][2], inner_table[i][3]])

  columns.clear()
  print(table)
  return inner_table

#таблица выходов
def outputTable(p_inp, p_st, x, v):
  inner_table = [[(str(y1(p_st[i])) + str(y2(p_st[i]))
                  + str(y3(p_st[i])) + str(y4(p_st[i]))) for j in range(len(p_inp))] for i in range(len(p_st))]

  print('Таблица выходов для автомата Мили')
  table = PrettyTable()
  columns = p_st.copy()
  columns.insert(0, "v \ x")
  table.field_names = columns

  for i in range(len(p_inp)):
    table.add_row([p_inp[i], inner_table[i][0], inner_table[i][1],
                             inner_table[i][2], inner_table[i][3]])

  columns.clear()
  print(table)
  return inner_table

###Задача 1. Мили

In [41]:
x0 = [0, 1]

#2 входа - v
v = [0, 1]
#2 состояния - x1/x2
x = [0, 1]
#4 выхода - y1/y2/y3/y4

#+(сумма по модулю 22), ^(стрелка пирса), & и v(конъюнкция и дизъюнкция)
#|(штрих шеффера), '(отрицание)
#->(импликация) и ~(эквивалентность)

In [42]:
pos_inputs = []
for i in range(len(v)):
  for j in range(len(v)):
    pos_inputs.append(str(v[i]) + str(v[j]))
print('Возможные входы:', pos_inputs)

pos_states = []
for i in range(len(x)):
  for j in range(len(x)):
    pos_states.append(str(x[i]) + str(x[j]))
print('Возможные состояния:', pos_states)

Возможные входы: ['00', '01', '10', '11']
Возможные состояния: ['00', '01', '10', '11']


In [46]:
def x1(x, v):
  #v1 + (x2 | x1 + v2 v (v2 + (x2 ^ x2)))
  rezult = int(v[0]) ^ (schaeffer(int(x[1]), int(x[0])) ^ int(v[1]) or (int(v[1]) ^ pier(int(x[1]), int(x[1]))))
  return rezult

def x2(x, v):
  #v1 -> v1 | (x1 ' | v1 + x1 ^ v2)
  rezult = schaeffer(implic(int(v[0]), int(v[0])), pier(schaeffer(int(x[0]), not int(v[0])) ^ int(x[0]), int(v[1])))
  return rezult

def y1(x):
  #x1 v x2 + x2 | x1 ~ x1 ~ (x2 ' -> x1)
  rezult = equiv(equiv(schaeffer(int(x[0]) or int(x[1]) ^ int(x[1]), int(x[0])), int(x[0])), implic(int(x[1]), not int(x[0])))
  return rezult

def y2(x):
  #x1 ~ x2 ^ (x1 + x2 | (x2 ~ x2))
  rezult = pier(equiv(int(x[0]), int(x[1])), schaeffer(int(x[0]) ^ int(x[1]), equiv(int(x[1]), int(x[1]))))
  return rezult

def y3(x):
  #x2 & x2 | x2 + x2 ^ x2
  rezult = pier(schaeffer(int(x[1]) and int(x[1]), int(x[1])) ^ int(x[1]), int(x[1]))
  return rezult

def y4(x):
  #x1 v x1 ~ x2 & (x2 ^ x1 ^ x2)
  rezult = equiv(int(x[0]) or int(x[0]), int(x[1]) and pier(pier(int(x[1]), int(x[0])), int(x[1])))
  return rezult

In [47]:
truth_table = truthTable(pos_inputs, pos_states, x, v)

Таблица истинности для автомата Мили
+-------+----+----+----+----+
| v \ x | 00 | 01 | 10 | 11 |
+-------+----+----+----+----+
|   00  | 11 | 01 | 01 | 11 |
|   01  | 11 | 11 | 01 | 01 |
|   10  | 11 | 01 | 00 | 11 |
|   11  | 01 | 11 | 10 | 01 |
+-------+----+----+----+----+


In [48]:
output_table = outputTable(pos_inputs, pos_states, x, v)

Таблица выходов для автомата Мили
+-------+------+------+------+------+
| v \ x |  00  |  01  |  10  |  11  |
+-------+------+------+------+------+
|   00  | 0001 | 0001 | 0001 | 0001 |
|   01  | 0101 | 0101 | 0101 | 0101 |
|   10  | 0100 | 0100 | 0100 | 0100 |
|   11  | 1000 | 1000 | 1000 | 1000 |
+-------+------+------+------+------+


###Задача 2. Мура

In [49]:
x0_2 = [0, 0]
#2 входа - v
#v = [0, 1]
#2 состояния - x
#4 выхода - y

In [51]:
def x1(x, v):
  #v1 -> (v2 & (x2 v v1 v v1 ^ x2 + (v2 -> x1 ~v2)))
  rezult = implic(int(v[0]), int(v[1]) and (pier(int(x[1]) or int(v[0]) or int(v[0]), int(x[1])) ^ equiv(implic(int(v[1]), int(x[0])), int(v[1]))))
  return rezult

def x2(x, v):
  #x1 | (x2 ^ v1 ' | (v1 ~ x1 + (x1 v v2)))
  rezult = schaeffer(int(x[0]), schaeffer(pier(int(x[1]), not int(v[0])), (equiv(int(v[0]), int(x[0])) ^ (int(x[0]) or int(v[1])))))
  return rezult

def y1(x):
  #x1 + v1 -> (x2 ^ v1 | (x1'))
  rezult = int(x[0]) ^ implic(v[0], pier(x[1], schaeffer(v[0], not x[0])))
  return rezult

def y2(x):
  #x2 v v2' -> (x2~v1)
  rezult = implic(x[1] and (not v[1]), equiv(x[1], v[0]))
  return rezult

def y3(x):
  #x1 & (x1 v (x2 & (x2 ^ (x1 & x2 + x1 | v2 & x1))))
  rezult = x[0] and (x[0] or (x[1] and pier(x[1], schaeffer(x[0] and x[1] ^ x[0], v[1] and x[0]))))
  return rezult

def y4(x):
  #v1 ^ x1' v (x1 v x2 -> x2 & (x2'))
  rezult = pier(v[0], not x[0]) or (implic(x[0] or x[1], x[1] and (not x[1])))
  return rezult

In [52]:
truth_table2 = truthTable(pos_inputs, pos_states, x, v)

Таблица истинности для автомата Мили
+-------+----+----+----+----+
| v \ x | 00 | 01 | 10 | 11 |
+-------+----+----+----+----+
|   00  | 11 | 11 | 01 | 01 |
|   01  | 11 | 11 | 01 | 01 |
|   10  | 10 | 10 | 00 | 10 |
|   11  | 10 | 10 | 00 | 10 |
+-------+----+----+----+----+


In [53]:
output_table = outputTable(pos_inputs, pos_states, x, v)

Таблица выходов для автомата Мили
+-------+------+------+------+------+
| v \ x |  00  |  01  |  10  |  11  |
+-------+------+------+------+------+
|   00  | 1101 | 1101 | 1101 | 1101 |
|   01  | 1101 | 1101 | 1101 | 1101 |
|   10  | 0111 | 0111 | 0111 | 0111 |
|   11  | 0111 | 0111 | 0111 | 0111 |
+-------+------+------+------+------+


##СКНФ/СДНФ
